In [1]:
import sys

sys.path.append("..") if ".." not in sys.path else print("Already exists")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms.functional as F
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from torchvision import models
from torchvision import transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from tqdm.notebook import tqdm

from utils.dataset import get_dataloader
from utils.training import TrainingInterface, print_total_params, train_network
from utils.plots import EvaluationPlots

### Initiate Dataloaders

In [2]:
further_transforms = transforms.Compose(
    [
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225]),
    ]
)

BATCH_SIZE = 32
WORKERS = 12

In [3]:
dataloader_train = get_dataloader(
    root_dir="../data/interim_train/",
    df=pd.read_csv("../data/train.csv"),
    fp_label_translator='../utils/dataset/label_translate.pkl',
    transformations=further_transforms,
    batch_size=BATCH_SIZE,
    workers=WORKERS,
    pin_memory=True,
    shuffle=True
)
dataloader_val = get_dataloader(
    root_dir="../data/val/",
    df=pd.read_csv("../data/val.csv"),
    fp_label_translator='../utils/dataset/label_translate.pkl',
    transformations=further_transforms,
    batch_size=BATCH_SIZE,
    workers=12,
    pin_memory=True,
    shuffle=True
)
dataloader_test = get_dataloader(
    root_dir="../data/test/",
    df=pd.read_csv("../data/test.csv"),
    fp_label_translator='../utils/dataset/label_translate.pkl',
    transformations=further_transforms,
    batch_size=BATCH_SIZE,
    workers=WORKERS,
    pin_memory=True,
    shuffle=True
)

In [4]:
assert dataloader_train.dataset.label_dict == dataloader_val.dataset.label_dict
assert dataloader_val.dataset.label_dict == dataloader_test.dataset.label_dict

### Check Dataloader Batch

In [5]:
# Plot batch
check_batch = False

plt.rcParams["savefig.bbox"] = "tight"

def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False, figsize=(12, 20))
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])


if check_batch:
    for loader in [dataloader_train, dataloader_val, dataloader_test]:
        img, labels = next(iter(loader))
        assert img.shape[0] == BATCH_SIZE
        assert len(labels) == BATCH_SIZE

# ConvNext

## Transfer-Learning

In [19]:
convnext_tiny_ = models.convnext_tiny(pretrained=True, progress=True)

In [20]:
#print(convnext_tiny_)

In [21]:
for param in convnext_tiny_.parameters():
    param.requires_grad = False

convnext_tiny_.classifier[2] = nn.Linear(
    in_features=768, out_features=len(dataloader_train.dataset.label_dict.keys())
)

for param in convnext_tiny_.classifier.parameters():
    param.requires_grad = False
    print(param.requires_grad)

# Set requires grad for FC weights
convnext_tiny_.classifier[2].weight.requires_grad = True
convnext_tiny_.classifier[2].bias.requires_grad = True

False
False
False
False


In [22]:
# Check
for name, param in convnext_tiny_.named_parameters():
    if param.requires_grad:
        print(name)

classifier.2.weight
classifier.2.bias


In [23]:
convnext = TrainingInterface(model=convnext_tiny_, name="ConvNext Tiny")

In [24]:
convnext.print_total_params()

ConvNext Tiny | Trainable Parameters: 27830125


In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(convnext.model.parameters(), lr=0.0001)

In [ ]:
convnext.train(
    criterion=criterion,
    optimizer=optimizer,
    n_epochs=15,
    dataloader_train=dataloader_train,
    dataloader_val=dataloader_val,
    verbose=True
)

  0%|          | 0/2970 [00:00<?, ?it/s]

Epoch 1/15: [Train-Loss = 430.98] || [Validation-Loss = 14.635]
Epoch 2/15: [Train-Loss = 326.69] || [Validation-Loss = 13.33]
Epoch 3/15: [Train-Loss = 277.042] || [Validation-Loss = 12.486]
Epoch 4/15: [Train-Loss = 247.56] || [Validation-Loss = 12.034]
Epoch 5/15: [Train-Loss = 228.793] || [Validation-Loss = 11.787]
Epoch 6/15: [Train-Loss = 214.51] || [Validation-Loss = 11.494]
Epoch 7/15: [Train-Loss = 203.826] || [Validation-Loss = 11.02]
Epoch 8/15: [Train-Loss = 194.113] || [Validation-Loss = 10.753]
Epoch 9/15: [Train-Loss = 186.82] || [Validation-Loss = 10.462]
Epoch 10/15: [Train-Loss = 180.605] || [Validation-Loss = 10.436]
Epoch 11/15: [Train-Loss = 175.976] || [Validation-Loss = 10.248]
Epoch 12/15: [Train-Loss = 171.029] || [Validation-Loss = 9.901]
Epoch 13/15: [Train-Loss = 167.015] || [Validation-Loss = 10.028]
Epoch 14/15: [Train-Loss = 163.088] || [Validation-Loss = 9.825]


In [ ]:
EvaluationPlots.plot_train_val_loss(convnext.train_loss, 
                                    convnext.val_loss)

In [ ]:
metrics = convnext.calculate_metrics(dataloader_train=dataloader_train, 
                                     dataloader_test=dataloader_test, 
                                     metric_funcs=[precision_score, recall_score, f1_score], 
                                     average='macro')

In [ ]:
metrics

In [ ]:
y_true, y_pred = convnext.predict(dataloader_test)

In [ ]:
EvaluationPlots.plot_confusion_matrix(y_true, y_pred, dataloader_test.dataset.label_dict_r)

In [ ]:
colors = sns.color_palette('Paired', 4)
sns.set_style('white')

fig = plt.subplots(figsize=(18, 5))

sma = 50

plt.subplot(1,2,1)
mean_loss_folds = losses_adam.mean(axis=1)
mean_loss_folds = mean_loss_folds.rolling(sma).mean()
std_loss_folds = losses_adam.std(axis=1)
std_loss_folds = std_loss_folds.rolling(sma).mean()

p = sns.lineplot(x=mean_loss_folds.index, y=mean_loss_folds, label='Mean Batch', color=colors[1])
p = sns.lineplot(x=mean_loss_folds.index, y=mean_loss_folds + std_loss_folds, 
                 label=r'$\pm1\sigma$', color=colors[0], linestyle='--', alpha=.5)
p = sns.lineplot(x=mean_loss_folds.index, y=mean_loss_folds - std_loss_folds, 
                 color=colors[0], linestyle='--', alpha=.5)
plt.text(x=mean_loss_folds.index[-1], y=mean_loss_folds.iloc[-1], 
         s=str(round(mean_loss_folds.iloc[-1], 2)), va='center')
p.set_title(f'Loss over Batches Adam-Optimizer / SMA{50}',loc='left')
p.set_xlabel('Batches')
p.set_ylabel('Cross-Entropy Loss')
sns.despine()

In [ ]:
def visualize_model(self, model, dataloader, num_images=6, ncol=3, 
                    label_translator=label_translator):
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    invTrans = Compose([ Normalize(mean = [ 0., 0., 0. ],
                                   std = [ 1/0.229, 1/0.224, 1/0.225 ]),
                        Normalize(mean = [ -0.485, -0.456, -0.406 ],
                                  std = [ 1., 1., 1. ]),
                       ])
    was_training = model.training
    model.eval()
    images_so_far = 0

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            fig = plt.subplots(figsize=(18, (num_images//ncol) * 5))
            for j in range(num_images):
                plt.subplot(num_images//ncol+1, ncol, j+1)
                plt.title('predicted: {} / actual: {}'.format(label_translator[preds[j].item()],
                                                              label_translator[labels[j].item()]))
                img = inputs.cpu().data[j]
                img = invTrans(img)
                img = to_pil_image(img)
                plt.imshow(img)
                images_so_far += 1

            model.train(mode=was_training)
            plt.subplots_adjust(hspace=.3)
            plt.show()
            break

## No Transfer-Learning

In [ ]:
convnext_tiny_ = models.convnext_tiny(pretrained=False, progress=True)

In [ ]:
for param in convnext_tiny_.parameters():
    param.requires_grad = True

In [ ]:
convnext_nopre = TrainingInterface(model=convnext_tiny_, name='ConvNext Tiny Not Pretrained') 

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(convnext.model.parameters(), lr=0.000005)

In [ ]:
convnext.train(
    criterion=criterion,
    optimizer=optimizer,
    n_epochs=5,
    dataloader_train=dataloader_train,
    dataloader_val=dataloader_val,
    verbose=True
)

In [ ]:
EvaluationPlots.plot_train_val_loss(convnext.train_loss, 
                                    convnext.val_loss,)

## AlexNet

### Pretrained

In [ ]:
alexnet_ = models.alexnet(pretrained=True, progress=True)

In [ ]:
alexnet_.classifier[6] = nn.Linear(4096, len(dataloader_train.dataset.label_dict.keys()))

In [ ]:
# Switch off grads for all layers
for param in alexnet_.parameters():
    param.requires_grad = False

In [ ]:
# Switch on gradients for classifer
for name, param in alexnet_.named_parameters():
    if 'classifier' in name:
        param.requires_grad = True
        print(name)

In [ ]:
alexnet = TrainingInterface(model=alexnet_, name='Alex Net')

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.Adam(alexnet.model.parameters(), lr=0.00000005)

In [ ]:
alexnet.train(
    criterion=criterion,
    optimizer=optimizer,
    n_epochs=5,
    dataloader_train=dataloader_train,
    dataloader_val=dataloader_val,
    verbose=True
)
EvaluationPlots.plot_train_val_loss(alexnet.train_loss, 
                                    alexnet.val_loss)

In [ ]:
alexnet.

### Not Pretrained

In [ ]:
alexnet_ = models.alexnet(pretrained=False, progress=True)

In [ ]:
# Switch off grads for all layers
for param in alexnet_.parameters():
    param.requires_grad = True

In [ ]:
# Exchange last layer 
alexnet_.classifier[6] = nn.Linear(4096, len(dataloader_train.dataset.label_dict.keys()))

In [ ]:
alexnet_np = TrainingInterface(model=alexnet_, name='Alex Net NP')

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.Adam(alexnet_np.model.parameters(), lr=0.000005)

In [ ]:
alexnet_np.train(
    criterion=criterion,
    optimizer=optimizer,
    n_epochs=5,
    dataloader_train=dataloader_train,
    dataloader_val=dataloader_val,
    verbose=True
)
EvaluationPlots.plot_train_val_loss(alexnet_np.train_loss, 
                                    alexnet_np.val_loss )

## ResNet18

### Pretrained

In [ ]:
resnet_ = models.resnet18(pretrained=True, progress=True)

In [ ]:
#print(resnet_)

In [ ]:
for param in resnet_.parameters():
    param.requires_grad = False

# Replace fc
resnet_.fc = nn.Linear(512, len(dataloader_train.dataset.label_dict.keys()))

# Enable grad
resnet_.fc.weight.requires_grad = True
resnet_.fc.bias.requires_grad = True

# check
for name, param in resnet_.named_parameters():
    if param.requires_grad:
        print('Requires Grad:', name)

In [ ]:
resnet = TrainingInterface(model=resnet_, name='ResNet18 pretrained')

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.model.parameters(), lr=.00001)

In [ ]:
resnet.train(
    criterion=criterion,
    optimizer=optimizer,
    n_epochs=10,
    dataloader_train=dataloader_train,
    dataloader_val=dataloader_val,
    verbose=True
)
EvaluationPlots.plot_train_val_loss(resnet.train_loss, resnet.val_loss)

In [ ]:
y_true, y_pred = resnet.predict(dataloader_test)

In [ ]:
EvaluationPlots.plot_confusion_matrix(y_true, y_pred, 
                                      class_labels=dataloader_test.dataset.label_dict_r)

In [ ]:
A = [1,2,4,123,2,1]

In [ ]:
list(zip(A))